In [22]:
import pandas as pd
import numpy 
import regex as re
import random
import sklearn


import scipy



In [23]:
pheno = pd.read_table("pheno.txt")
data = pd.read_table("normalized_data.txt")

pheno.head()


,ParticipantID,Phenotype,Sample_Source,PlateId,Sex,HybControlNormScale,RowCheck,NormScale_20,NormScale_0_005,NormScale_0_5,ANMLFractionUsed_20,ANMLFractionUsed_0_005,ANMLFractionUsed_0_5
0,C1,Control,Blood,P0023923,F,0.947480,PASS,0.769958,0.813128,0.818578,0.868,0.838,0.745
1,C2,Control,Blood,P0023895,F,0.891341,PASS,0.765640,0.967612,0.891220,0.874,0.900,0.790
2,C3,Control,Blood,P0023895,F,0.888196,PASS,0.412029,0.918688,0.810701,0.732,0.881,0.666
3,C4,Control,Blood,P0023923,F,0.856026,PASS,0.774548,0.866460,0.810048,0.879,0.875,0.777
4,C5,Control,Blood,P0023895,F,0.982222,PASS,0.665371,0.833492,0.820785,0.804,0.900,0.852


In [24]:
data.head()

metadata = data.iloc[:, :4]

metadata

,Molecule,Protein,Gene,UniProt
0,10000-28,CRBB2,CRYBB2,P43320
1,10001-7,c-Raf,RAF1,P04049
2,10003-15,ZNF41,ZNF41,P51814
3,10006-25,ELK1,ELK1,P19419
4,10008-43,GUC1A,GUCA1A,P43080
...,...,...,...,...
5279,9993-11,ZN264,ZNF264,O43296
5280,9994-217,ATP4B,ATP4B,P51164
5281,9995-6,DUT,DUT,P33316
5282,9997-12,UBXN4,UBXN4,Q92575


In [25]:
data = data.transpose()



In [26]:
data.columns = data.iloc[0]
data.drop(data.index[0:4], inplace=True)





In [30]:
target = data.index.tolist()

#replace controls with 0 and patient with 1

target = [0 if re.search('[C]', x) else 1 for x in target ]


In [31]:
data["target"] = target
data['target']

C1     0
C2     0
C3     0
C4     0
C5     0
C8     0
C9     0
C11    0
C14    0
C15    0
C16    0
C17    0
C18    0
C19    0
C20    0
C21    0
C24    0
C25    0
C26    0
C27    0
P1     1
P2     1
P4     1
P6     1
P9     1
P11    1
P13    1
P17    1
P18    1
P19    1
P20    1
P21    1
P24    1
P25    1
P27    1
P28    1
P29    1
P30    1
P31    1
P32    1
Name: target, dtype: int64

### Fitting GLM

In [37]:
from sklearn.linear_model import LogisticRegression

X = data.drop(["target"],axis=1)
y = data['target']

clf = LogisticRegression()
clf.fit(X,y)

clf.coef_

array([[5.24671850e-05, 1.23688370e-07, 4.24787073e-08, ...,
        1.84657639e-05, 1.18166631e-05, 2.05128490e-06]])

In [9]:
cases = data[data["target"]=="Case"]
controls = data[data["target"]!="Case"]


def simulate_protein_data(df):
    means = df.mean(axis=0).tolist()
    sds = df.std(axis=0).tolist()

    if len(means) != len(sds): ## sanity check
        return "List Imbalance"
    
    simulated_data = []
    for i in range(len(means)):
        col = []
        for _ in range((1000)):
            col.append(random.gauss(means[i],sds[i]))
        simulated_data.append(col)
        

    simulated_dataframe = pd.DataFrame(simulated_data)
    return simulated_dataframe 




In [10]:
cases_simul = simulate_protein_data(cases)

/var/folders/yj/znfgbkgj4f1gvy__jfl3rw6c0000gn/T/ipykernel_4562/336261786.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  means = df.mean(axis=0).tolist()
/var/folders/yj/znfgbkgj4f1gvy__jfl3rw6c0000gn/T/ipykernel_4562/336261786.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  sds = df.std(axis=0).tolist()


In [113]:
cases_simul

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,8334.217918,15072.688985,3922.138778,1633.791290,-2590.013169,7053.105447,-1539.653744,-3311.676133,11819.928607,-16755.456791,...,6014.038696,-1443.315256,3161.517565,15753.056642,5576.299950,5475.473060,12280.039414,14103.681075,344.288393,324.798358
1,248.706004,267.486169,272.785676,287.504725,173.241579,216.558338,228.824812,211.371410,223.870291,208.043209,...,220.121159,225.676685,212.704080,244.827548,168.954258,242.559165,285.722037,200.842513,235.087296,226.222470
2,98.585963,147.262624,76.114877,128.766304,112.747871,155.527671,126.142614,-33.501536,-23.838260,-128.721032,...,283.768349,224.636131,57.799425,381.234330,-148.638721,136.141746,269.711737,232.969882,295.944678,229.453387
3,1317.570612,1975.024242,129.341223,142.268494,699.371529,537.653463,1198.054640,828.036901,1505.475510,1120.426955,...,1768.038110,1531.242452,500.466676,1344.665332,1179.482671,515.451689,863.797658,733.972598,1088.382014,1512.896956
4,432.963741,472.295452,408.891090,529.144088,475.280567,615.141049,463.589668,566.451140,535.257246,596.863478,...,366.506273,542.818530,600.169346,521.841979,487.522118,504.500491,531.083260,502.997161,493.142163,567.518730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5279,943.144158,925.148804,999.098764,936.640762,975.261623,939.241526,954.242762,1015.618551,906.042185,1028.604293,...,1001.793540,1027.977605,969.866723,902.185022,936.877349,1001.894603,901.265342,1039.149707,1010.187522,1073.791850
5280,8962.379246,9122.403800,13154.452786,10914.819767,115.068199,14535.323578,16643.952881,8238.136815,3454.127795,1341.242917,...,6423.377584,13498.819835,12026.845140,629.840518,-1154.038115,5666.387960,10853.943206,4019.609000,16629.708065,4459.612843
5281,1239.332904,5428.903803,4636.139394,3412.581616,-4276.382197,-2462.182266,168.439642,5547.735757,2854.296669,2354.556349,...,5410.451716,-1005.540574,2596.290598,5802.001753,2530.711202,4332.907089,1886.677745,2249.585962,7027.182679,8128.055261
5282,16870.038273,5562.826465,2438.880935,10031.513966,11031.770788,5391.403300,10224.815110,7821.346307,7477.350676,8827.703978,...,9370.152799,7714.995680,20964.942899,6122.097657,19967.204201,5555.968158,15366.061792,11142.898122,13540.371085,2856.684355


In [100]:
cases.mean(axis=0).tolist()

/var/folders/yj/znfgbkgj4f1gvy__jfl3rw6c0000gn/T/ipykernel_57915/2859408631.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  cases.mean(axis=0).tolist()[0]


2587.92

 ## Data Simulation Time

In [85]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(data.drop(['target'], axis = 1), data["target"] ,test_size=0.25,random_state=0)

In [86]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# Load the dataset

# Create a logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)


In [87]:
from sklearn import metrics

cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[1, 4],
       [1, 4]])

array(['Control', 'Control', 'Case', 'Case', 'Control', 'Control',
       'Control', 'Control', 'Control', 'Control'], dtype=object)